In [1]:
# 2クラス分類の結果をマージする
import numpy as np
import pandas as pd
import sys, os
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [7]:
# ennのパス
enn_filedir = os.path.join(os.environ["git"],"sleep_study", "bin_enn", "*.csv")
# cnn+attnのパス
cnn_attn_filedir = os.path.join(os.environ["git"],"sleep_study", "cnn_attn", "*.csv")
# cnnのパス
cnn_filedir = os.path.join(os.environ["git"],"sleep_study", "cnn_no_attn", "*.csv")

In [8]:
enn_filelist = glob(enn_filedir)
cnn_attn_filelist = glob(cnn_attn_filedir)
cnn_filelist = glob(cnn_filedir)

In [9]:
print(enn_filelist[0], 
      cnn_attn_filelist[0], 
      cnn_filelist[0])

C:\Users\taiki\git\sleep_study\bin_enn\output_140703_Li.csv C:\Users\taiki\git\sleep_study\cnn_attn\140703_Li.csv C:\Users\taiki\git\sleep_study\cnn_no_attn\output_0_iei_202101.csv


In [45]:
# tmp
df = pd.read_csv(enn_filelist[0])
df
# df.iloc[:, 1]
# ilocは行列をインデックスで指定したいときのみ使用するようにする
# df.loc[:, ["nr1_pred", "nr2_pred", "nr34_pred", "rem_pred", "wake_pred", "y_true"]]

,Unnamed: 0,time,nr1_pred,nr2_pred,nr34_pred,rem_pred,wake_pred,nr1_unc,nr2_unc,nr34_unc,rem_unc,wake_unc,y_true
0,0,00:28:04,0,0,0,1,0,1.0,0.022223,0.012790,0.027827,1.00000,1
1,1,00:28:34,0,0,0,0,0,1.0,0.028135,0.013394,0.062719,1.00000,1
2,2,00:29:04,0,0,0,1,0,1.0,0.025857,0.013822,0.094508,1.00000,1
3,3,00:29:34,0,0,0,1,0,1.0,0.027468,0.014498,0.059256,1.00000,1
4,4,00:30:04,0,0,0,0,0,1.0,0.024136,0.014692,0.073933,0.87598,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,901,07:58:35,0,0,0,0,0,1.0,0.116518,0.073631,0.038378,1.00000,4
902,902,07:59:05,0,0,1,0,0,1.0,0.116266,0.112624,0.041456,1.00000,4
903,903,07:59:35,0,0,0,0,0,1.0,0.102047,0.065078,0.064603,1.00000,4
904,904,08:00:05,0,0,0,0,0,1.0,0.076440,0.036687,0.022374,1.00000,4


In [66]:
# 統合ルールA
# 0. フラグが一本立っている場合はそれ
# 1. フラグが複数立っている場合一番不確実性の一番低いものを採用する
# 2. フラグが何も立っていない場合はNR2と判断する
def merge_rule_a(series: pd.Series):
    bin_pred = series[
        ["nr34_pred", "nr2_pred", "nr1_pred", "rem_pred", "wake_pred"]
    ]
    bin_pred = bin_pred.to_numpy()

    if sum(bin_pred) == 1:
        return np.argmax(bin_pred)

    elif sum(bin_pred) > 1:
        flag_unc = series[
            ["nr34_unc", "nr2_unc", "nr1_unc", "rem_unc", "wake_unc"]
        ]
        # フラグが立っている場所だけ残す
        flag_unc = flag_unc.to_numpy() * bin_pred
        # 不確実性が最小の睡眠段階を抽出する
        #             print(flag_unc == np.min(flag_unc[np.nonzero(flag)]))
        #             print(np.argmax(flag_unc == np.min(flag_unc[np.nonzero(flag)])))
        return np.argmax(flag_unc == np.min(flag_unc[np.nonzero(bin_pred)]))

    else:
        assert sum(bin_pred) == 0
        # NR2を返す
        return 1

# 統合ルールB
# 0. ひとつだけPositiveならそのクラス
# 1. NR2, remがPositiveならrem
# 2. nr1, wakeがpositiveならwake
# 3. positiveのなかで優先度（）の高いもので並び替え
# 4. nr2
def merge_rule_b(series: pd.Series):
    bin_pred = series[
        ["nr34_pred", "nr2_pred", "nr1_pred", "rem_pred", "wake_pred"]
    ]
    bin_pred = bin_pred.to_numpy()
    # 0. ひとつだけPositiveならそのクラス
    if sum(bin_pred) == 1:
        return np.argmax(bin_pred)

    # 1. NR2, remがPositiveならrem
    elif bin_pred[2] == 1 and bin_pred[4] == 1:
        return 3

    # 2. nr1, wakeがpositiveならwake
    elif bin_pred[2] == 1 and bin_pred[4] == 1:
        return 4

    # 3. positiveのなかで優先度の高いもので並び替え
    elif int(sum(bin_pred)) > 1:
        if bin_pred[1] == 1:
            return 1
        elif bin_pred[3] == 1:
            return 3
        elif bin_pred[2] == 1:
            return 2
        elif bin_pred[4] == 1:
            return 4
        elif bin_pred[0] == 1:
            return 0
        else:
            raise AssertionError

def calc_acc(df):
    def __judge_acc_from_column_id(l):
        return sum(df[str(l[0])] == df[str(l[1])])
    acc_list = list(map(__judge_acc_from_column_id, [[11, 12], [11, 13]]))
    print("統合ルールA, Bの正解数", acc_list)
    row, _ = df.shape
    return [_acc_list / row for _acc_list in acc_list]

In [67]:
df.apply(merge_rule_a, axis=1)

ValueError: key of type tuple not found and not a MultiIndex